<a href="https://colab.research.google.com/github/thamyrissacamoto/MarvelAPI/blob/Thamyris%2FconsumindoAPI/client.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import hashlib
import time
import requests
from google.colab import userdata

public_key=userdata.get('public_key')
private_key=userdata.get('private_key')

ts = str(time.time())
to_hash = ts + private_key + public_key
hash_result = hashlib.md5(to_hash.encode()).hexdigest()

url = "https://gateway.marvel.com/v1/public/characters"

params = {
    "ts": ts,
    "apikey": public_key,
    "hash": hash_result,
    "limit": 5
}

response = requests.get(url, params=params)

print(response.status_code)
print(response.json())

200
{'code': 200, 'status': 'Ok', 'copyright': '© 2025 MARVEL', 'attributionText': 'Data provided by Marvel. © 2025 MARVEL', 'attributionHTML': '<a href="http://marvel.com">Data provided by Marvel. © 2025 MARVEL</a>', 'etag': 'be7e21418f46556bc56bed7d4db9d958363042f9', 'data': {'offset': 0, 'limit': 5, 'total': 1564, 'count': 5, 'results': [{'id': 1011334, 'name': '3-D Man', 'description': '', 'modified': '2014-04-29T14:18:17+0000', 'thumbnail': {'path': 'http://i.annihil.us/u/prod/marvel/i/mg/c/e0/535fecbbb9784', 'extension': 'jpg'}, 'resourceURI': 'https://gateway.marvel.com/v1/public/characters/1011334', 'comics': {'available': 12, 'collectionURI': 'https://gateway.marvel.com/v1/public/characters/1011334/comics', 'items': [{'resourceURI': 'https://gateway.marvel.com/v1/public/comics/24571', 'name': 'Avengers: The Initiative (2007) #14'}, {'resourceURI': 'https://gateway.marvel.com/v1/public/comics/21366', 'name': 'Avengers: The Initiative (2007) #14'}, {'resourceURI': 'https://gatew